In [ ]:
class KRLS:
    def __init__(self, kernel_type='rbf', sigma=1.0, c=1.0, nu=1e-6):
        """
        Inicializa o modelo KRLS.
        
        :param kernel_type: Tipo de kernel a ser utilizado ('rbf' ou 'linear')
        :param sigma: Parâmetro do kernel RBF
        :param c: Parâmetro de regularização
        :param nu: Critério de crescimento
        """
        self.kernel_type = kernel_type  # Tipo de kernel ('rbf' ou 'linear')
        self.sigma = sigma  # Parâmetro do kernel RBF
        self.c = c          # Parâmetro de regularização
        self.nu = nu        # Critério de crescimento
        self.D = []         # Dicionário de vetores de suporte
        self.alpha = []     # Coeficientes de expansão
        self.K_inv = []     # Inversa da matriz de kernel regularizada

    def __rbf_kernel(self, x1, x2):
        """
        Calcula o kernel RBF entre dois vetores (função privada).
        """
        return np.exp(-np.linalg.norm(x1 - x2) ** 2 / (2 * self.sigma ** 2))
    
    def __linear_kernel(self, x1, x2):
        """
        Calcula o kernel linear entre dois vetores (função privada).
        """
        return np.dot(x1, x2)

    def __kernel(self, x1, x2):
        """
        Escolhe o kernel de acordo com o tipo especificado ('rbf' ou 'linear').
        """
        if self.kernel_type == 'rbf':
            return self.__rbf_kernel(x1, x2)
        elif self.kernel_type == 'linear':
            return self.__linear_kernel(x1, x2)
        else:
            raise ValueError("Kernel type not supported. Choose 'rbf' or 'linear'.")

    def predict(self, x):
        """
        Prediz o valor da saída para uma nova amostra x, usando o dicionário atual e os coeficientes.
        """
        if not self.D:  # Se o dicionário estiver vazio
            return 0
        k_n = np.array([self.__kernel(x, u) for u in self.D])
        return k_n.T @ self.alpha

    def __compute_an(self, k_n):
        """
        Calcula o vetor intermediário a_n (função privada).
        """
        return self.K_inv @ k_n

    def __compute_gamma_n(self, k_nn, k_n, a_n):
        """
        Calcula o valor escalar gamma_n (função privada).
        """
        return k_nn + self.c - k_n.T @ a_n

    def __update_kernel_inverse(self, a_n, gamma_n):
        """
        Atualiza a inversa da matriz de kernel quando o dicionário é expandido (função privada).
        """
        K_inv_old = self.K_inv
        K_inv_new = np.zeros((len(self.D), len(self.D)))
        K_inv_new[:-1, :-1] = K_inv_old + np.outer(a_n, a_n) / gamma_n
        K_inv_new[:-1, -1] = -a_n / gamma_n
        K_inv_new[-1, :-1] = -a_n.T / gamma_n
        K_inv_new[-1, -1] = 1 / gamma_n
        return K_inv_new

    def __update_alpha(self, e_n, gamma_n, a_n):
        """
        Atualiza os coeficientes de expansão alpha (função privada).
        """
        self.alpha = np.append(self.alpha, 0)
        self.alpha += (e_n / gamma_n) * np.append(-a_n, 1)

    def update(self, x_n, y_n):
        """
        Atualiza o modelo KRLS com a nova amostra (x_n, y_n).
        """
        if not self.D:
            # Inicializa o dicionário e a matriz K_inv
            self.D = [x_n]
            self.K_inv = np.array([[1 / (self.__kernel(x_n, x_n) + self.c)]])
            self.alpha = np.array([y_n])
            return

        # Predição
        y_hat = self.predict(x_n)

        # Cálculo do erro
        e_n = y_n - y_hat

        # Vetor de kernel k_n
        k_n = np.array([self.__kernel(x_n, u) for u in self.D])

        # Similaridade com o próprio ponto
        k_nn = self.__kernel(x_n, x_n)

        # Cálculo de a_n
        a_n = self.__compute_an(k_n)

        # Cálculo de gamma_n
        gamma_n = self.__compute_gamma_n(k_nn, k_n, a_n)

        # Critério de crescimento do dicionário
        if gamma_n > self.nu:
            # Expandir o dicionário
            self.D.append(x_n)

            # Atualizar a matriz inversa de kernel
            self.K_inv = self.__update_kernel_inverse(a_n, gamma_n)

            # Atualizar os coeficientes de expansão alpha
            self.__update_alpha(e_n, gamma_n, a_n)
        else:
            # Atualizar o dicionário sem expansão
            self.alpha += (e_n / gamma_n) * a_n
# Exemplo de uso
krls = KRLS(kernel_type='linear', sigma=1.0, c=10.0, nu=0.5)

In [1]:
from sklearn.kernel_ridge import KernelRidge
import numpy as np